# Succinate production landscape under environmental variation

This notebook builds on the succinate hands-on session to focus specifically on
succinate secretion as the primary optimisation target. We capture co-produced
organic acids, inspect how feed supplements shift the succinate optimum, and map
the surrounding TCA/glyoxylate reactions that support high succinate flux.

![Succinate pathway reference](media/succinate_pathway_reference.png)


## 1. Prerequisites

* Create and activate a Python environment with COBRApy and plotting tools:

```bash
python -m venv .venv
source .venv/bin/activate
pip install cobra[glpk] pandas matplotlib seaborn networkx
```

* Ensure this notebook runs from the repository root (`yeast-GEM`).

In [ ]:
from pathlib import Path

# Ensure repository-specific helpers can locate configuration files
Path('.env').touch()
print('`.env` marker file present.')

In [ ]:
import itertools
from typing import Dict, Iterable, List, Tuple
import math
import warnings

import cobra
import pandas as pd
from cobra import Model
from cobra.flux_analysis import pfba
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from code.io import read_yeast_model

sns.set_theme(style='whitegrid')
warnings.filterwarnings(
    'ignore',
    message="Solver status is 'infeasible'.",
    category=UserWarning,
    module='cobra.util.solver',
)

## 2. Load the yeast9 model

In [ ]:
analysis_model: Model = read_yeast_model()

print(f'Reactions: {len(analysis_model.reactions)}')
print(f'Metabolites: {len(analysis_model.metabolites)}')
print(f'Genes: {len(analysis_model.genes)}')
print(f'Default objective: {analysis_model.objective.expression}')

## 3. Configure the Kennedy synthetic medium and feed supplements

In [ ]:
# Exchange reactions that receive a constrained uptake (-0.5 mmol gDW⁻¹ h⁻¹)
CONSTRAINED_UPTAKE: Iterable[str] = (
    'r_1604', 'r_1639', 'r_1873', 'r_1879', 'r_1880', 'r_1881', 'r_1671',
    'r_1883', 'r_1757', 'r_1891', 'r_1889', 'r_1810', 'r_1993', 'r_1893',
    'r_1897', 'r_1947', 'r_1899', 'r_1900', 'r_1902', 'r_1967', 'r_1903',
    'r_1548', 'r_1904', 'r_2028', 'r_2038', 'r_1906', 'r_2067', 'r_1911',
    'r_1912', 'r_1913', 'r_2090', 'r_1914', 'r_2106',
)

UNCONSTRAINED_UPTAKE: Iterable[str] = (
    'r_1672',  # ammonium
    'r_1654',  # potassium
    'r_1992',  # oxygen
    'r_2005',  # phosphate
    'r_2060',  # sulfate
    'r_1861',  # iron
    'r_1832',  # proton
    'r_2100',  # water
    'r_4593',  # chloride
    'r_4595',  # manganese
    'r_4596',  # zinc
    'r_4597',  # magnesium
    'r_2049',  # sodium
    'r_4594',  # copper
    'r_4600',  # calcium
    'r_2020',  # sulfate/glutathione coupled import
)

GLUCOSE_EXCHANGE_ID = 'r_1714'
OXYGEN_EXCHANGE_ID = 'r_1992'
PROTON_EXCHANGE_ID = 'r_1832'
AMMONIUM_EXCHANGE_ID = 'r_1672'

BIOMASS_REACTION_ID = 'r_2111'
NGAM_REACTION_ID = 'r_4046'
SUCCINATE_EXCHANGE_ID = 'r_2056'

ACID_EXCHANGES: Dict[str, str] = {
    'succinate': SUCCINATE_EXCHANGE_ID,
    'citrate': 'r_1687',
    'malate': 'r_1552',
    'lactate': 'r_1551',
    'formate': 'r_1793',
    'acetate': 'r_1634',
    'glutamate': 'r_1889',
}

FEED_PROFILES: Dict[str, Dict[str, float]] = {
    'none': {},
    'citrate_1pct': {'r_1687': -1.0},
    'citrate_3pct': {'r_1687': -3.0},
    'glutamate_0p5pct': {'r_1889': -0.5},
    'glutamate_1pct': {'r_1889': -1.0},
}

TEMPERATURE_NGAM = {
    '20C': 0.6,
    '30C': 0.7,
    '37C': 0.8,
}

DEFAULT_AMMONIUM_LIMIT = -10.0
DEFAULT_GLUCOSE_LIMIT = -20.0


def apply_kennedy_medium(model: Model, glucose_limit: float = DEFAULT_GLUCOSE_LIMIT) -> None:
    for rxn in model.exchanges:
        rxn.lower_bound = 0.0
        rxn.upper_bound = 1000.0
    for rxn_id in CONSTRAINED_UPTAKE:
        model.reactions.get_by_id(rxn_id).lower_bound = -0.5
    for rxn_id in UNCONSTRAINED_UPTAKE:
        model.reactions.get_by_id(rxn_id).lower_bound = -1000.0
    model.reactions.get_by_id(GLUCOSE_EXCHANGE_ID).lower_bound = glucose_limit


def apply_feed_profile(model: Model, profile: Dict[str, float]) -> None:
    for rxn_id, limit in profile.items():
        rxn = model.reactions.get_by_id(rxn_id)
        rxn.lower_bound = limit


## 4. Environmental evaluation helper

For each combination of feed label, oxygen/proton limits, and temperature tag we:

1. Apply the Kennedy medium and feed supplement.
2. Set environmental bounds (oxygen, proton, ammonium, non-growth associated maintenance).
3. Optimise for maximum succinate secretion.
4. Record the succinate optimum and co-produced organic acid fluxes.
5. Capture biomass and redox indicators for deeper inspection.

In [ ]:
REDOX_METABOLITES = {
    'NADH_m': 's_1205',
    'NAD_m': 's_0792',
    'FADH2_m': 's_0690',
    'FAD_m': 's_0688',
}

TCA_NEIGHBOUR_REACTIONS: Dict[str, Tuple[str, str]] = {
    'citrate_synthase': ('r_0432', 'Citrate synthase'),
    'aconitase': ('r_0108', 'Aconitase'),
    'isocitrate_dehydrogenase_NAD': ('r_0884', 'Isocitrate dehydrogenase (NAD⁺)'),
    'isocitrate_dehydrogenase_NADP': ('r_0885', 'Isocitrate dehydrogenase (NADP⁺)'),
    'akg_dehydrogenase': ('r_0118', 'α-Ketoglutarate dehydrogenase'),
    'succinyl_CoA_synthetase': ('r_1212', 'Succinyl-CoA synthetase'),
    'succinate_dehydrogenase': ('r_1216', 'Succinate dehydrogenase'),
    'fumarase': ('r_0765', 'Fumarase'),
    'malate_dehydrogenase': ('r_0952', 'Malate dehydrogenase'),
    'glyoxylate_shunt_1': ('r_0890', 'Isocitrate lyase'),
    'glyoxylate_shunt_2': ('r_0891', 'Malate synthase'),
}


def configure_environment(
    model: Model,
    *,
    oxygen_limit: float,
    proton_limit: float,
    ammonium_limit: float,
    temperature_tag: str,
) -> None:
    model.reactions.get_by_id(OXYGEN_EXCHANGE_ID).lower_bound = oxygen_limit
    model.reactions.get_by_id(PROTON_EXCHANGE_ID).lower_bound = proton_limit
    model.reactions.get_by_id(AMMONIUM_EXCHANGE_ID).lower_bound = ammonium_limit
    ngam = TEMPERATURE_NGAM[temperature_tag]
    ngam_rxn = model.reactions.get_by_id(NGAM_REACTION_ID)
    ngam_rxn.lower_bound = ngam
    ngam_rxn.upper_bound = ngam


def compute_redox_balances(model: Model, fluxes: pd.Series) -> Dict[str, float]:
    balances: Dict[str, float] = {}
    for label, met_id in REDOX_METABOLITES.items():
        metabolite = model.metabolites.get_by_id(met_id)
        total = 0.0
        for rxn in model.reactions:
            if metabolite in rxn.metabolites and rxn.id in fluxes.index:
                coeff = rxn.metabolites[metabolite]
                total += coeff * fluxes[rxn.id]
        balances[label] = total
    return balances


def evaluate_environment(
    model: Model,
    *,
    feed_label: str,
    glucose_limit: float,
    oxygen_limit: float,
    proton_limit: float,
    ammonium_limit: float,
    temperature_tag: str,
) -> Dict[str, float]:
    with model:
        apply_kennedy_medium(model, glucose_limit=glucose_limit)
        apply_feed_profile(model, FEED_PROFILES[feed_label])
        configure_environment(
            model,
            oxygen_limit=oxygen_limit,
            proton_limit=proton_limit,
            ammonium_limit=ammonium_limit,
            temperature_tag=temperature_tag,
        )
        model.objective = model.reactions.get_by_id(SUCCINATE_EXCHANGE_ID)
        solution = model.optimize()
        record: Dict[str, float] = {
            'feed_label': feed_label,
            'glucose_limit': glucose_limit,
            'oxygen_limit': oxygen_limit,
            'proton_limit': proton_limit,
            'ammonium_limit': ammonium_limit,
            'temperature_tag': temperature_tag,
            'status': solution.status,
        }
        if solution.status != 'optimal':
            return record
        fluxes = solution.fluxes
        record['succinate_flux'] = fluxes[SUCCINATE_EXCHANGE_ID]
        record['biomass_flux'] = fluxes.get(BIOMASS_REACTION_ID, float('nan'))
        for label, rxn_id in ACID_EXCHANGES.items():
            record[f'{label}_flux'] = fluxes.get(rxn_id, float('nan'))
        redox = compute_redox_balances(model, fluxes)
        for key, value in redox.items():
            record[f'redox_{key}'] = value
        for key, (_, pretty) in TCA_NEIGHBOUR_REACTIONS.items():
            record[f'reaction_{key}'] = fluxes.get(TCA_NEIGHBOUR_REACTIONS[key][0], float('nan'))
        return record


## 5. Run an environmental sweep

The helper below iterates across oxygen, proton, and temperature grids for each
feed profile. Adjust the tuples to explore alternative ranges.

In [ ]:
def run_environmental_sweep(
    model: Model,
    *,
    feed_labels: Iterable[str],
    glucose_grid: Iterable[float],
    oxygen_grid: Iterable[float],
    proton_grid: Iterable[float],
    temperature_grid: Iterable[str],
    ammonium_limit: float = DEFAULT_AMMONIUM_LIMIT,
) -> pd.DataFrame:
    records: List[Dict[str, float]] = []
    for feed_label, glucose_limit, oxygen_limit, proton_limit, temperature_tag in itertools.product(
        feed_labels, glucose_grid, oxygen_grid, proton_grid, temperature_grid
    ):
        record = evaluate_environment(
            model,
            feed_label=feed_label,
            glucose_limit=glucose_limit,
            oxygen_limit=oxygen_limit,
            proton_limit=proton_limit,
            ammonium_limit=ammonium_limit,
            temperature_tag=temperature_tag,
        )
        records.append(record)
    return pd.DataFrame(records)


def top_succinate_conditions(results: pd.DataFrame, top_n: int = 5) -> pd.DataFrame:
    feasible = results[results['status'] == 'optimal'].copy()
    if feasible.empty:
        return feasible
    columns = [
        'feed_label', 'glucose_limit', 'oxygen_limit', 'proton_limit',
        'ammonium_limit', 'temperature_tag', 'succinate_flux', 'biomass_flux',
    ] + [f'{label}_flux' for label in ACID_EXCHANGES if label != 'succinate']
    return feasible.sort_values('succinate_flux', ascending=False).head(top_n)[columns]


In [ ]:
glucose_grid = (-10.0, -15.0, -20.0)
oxygen_grid = (0.0, -5.0, -10.0)
proton_grid = (-1.0, -5.0, -10.0)
temperature_grid = ('20C', '30C', '37C')
feed_labels = tuple(FEED_PROFILES.keys())

sweep_results = run_environmental_sweep(
    analysis_model,
    feed_labels=feed_labels,
    glucose_grid=glucose_grid,
    oxygen_grid=oxygen_grid,
    proton_grid=proton_grid,
    temperature_grid=temperature_grid,
)
print(f"Total scenarios evaluated: {len(sweep_results)}")
sweep_results.head()


### Top succinate-producing scenarios

The table lists the best succinate fluxes and the accompanying by-product fluxes
for each candidate environment.

In [ ]:
top_succinate_conditions(sweep_results, top_n=10)

### Feed-specific champions

Identify, for each feed supplement, the condition that maximises succinate and
report the co-produced organic acid fluxes.

In [ ]:
def feed_specific_best(results: pd.DataFrame) -> pd.DataFrame:
    feasible = results[results['status'] == 'optimal'].copy()
    if feasible.empty:
        return feasible
    slices: List[pd.DataFrame] = []
    for feed_label, group in feasible.groupby('feed_label'):
        champion = group.sort_values('succinate_flux', ascending=False).head(1)
        columns = [
            'feed_label', 'succinate_flux', 'biomass_flux', 'glucose_limit',
            'oxygen_limit', 'proton_limit', 'temperature_tag',
        ] + [f'{label}_flux' for label in ACID_EXCHANGES if label != 'succinate']
        slices.append(champion[columns])
    return pd.concat(slices, ignore_index=True)

feed_specific_best(sweep_results)


## 6. Visualise succinate trade-offs

We can reshape the grid to inspect how oxygen and proton bounds influence
succinate secretion within a chosen feed supplement.

In [ ]:
def plot_succinate_heatmap(
    results: pd.DataFrame,
    *,
    feed_label: str,
    temperature_tag: str,
    glucose_limit: float,
) -> None:
    feasible = results[
        (results['status'] == 'optimal')
        & (results['feed_label'] == feed_label)
        & (results['temperature_tag'] == temperature_tag)
        & (results['glucose_limit'] == glucose_limit)
    ]
    if feasible.empty:
        raise ValueError('No feasible points for the requested slice.')
    pivot = feasible.pivot_table(
        index='oxygen_limit', columns='proton_limit', values='succinate_flux'
    )
    plt.figure(figsize=(6, 4))
    sns.heatmap(pivot, annot=True, fmt='.2f', cmap='magma', cbar_kws={'label': 'Succinate flux'})
    plt.title(f'Succinate flux (feed={feed_label}, T={temperature_tag}, glucose={glucose_limit})')
    plt.xlabel('Proton lower bound (mmol gDW⁻¹ h⁻¹)')
    plt.ylabel('Oxygen lower bound (mmol gDW⁻¹ h⁻¹)')
    plt.show()


def plot_growth_tradeoff(results: pd.DataFrame, *, feed_label: str) -> None:
    feasible = results[(results['status'] == 'optimal') & (results['feed_label'] == feed_label)]
    if feasible.empty:
        raise ValueError('No feasible points for the requested feed label.')
    plt.figure(figsize=(6, 4))
    sns.scatterplot(
        data=feasible,
        x='biomass_flux',
        y='succinate_flux',
        hue='temperature_tag',
        style='oxygen_limit',
        size='proton_limit',
        palette='viridis',
        sizes=(50, 200),
    )
    plt.title(f'Succinate vs biomass (feed={feed_label})')
    plt.xlabel('Biomass flux (mmol gDW⁻¹ h⁻¹)')
    plt.ylabel('Succinate flux (mmol gDW⁻¹ h⁻¹)')
    plt.legend(bbox_to_anchor=(1.04, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


In [ ]:
plot_succinate_heatmap(
    sweep_results,
    feed_label='none',
    temperature_tag='30C',
    glucose_limit=-20.0,
)


In [ ]:
plot_growth_tradeoff(sweep_results, feed_label='none')

## 7. Reaction-level interpretation

To map the reactions most connected to succinate secretion we reuse the
succinate neighbourhood utilities and apply pFBA to highlight high-flux routes.

In [ ]:
SUCCINATE_METABOLITE_IDS = {
    "s_1458",  # succinate [c]
    "s_1460",  # succinate [m]
    "s_1459",  # succinate [e]
}

KEY_SUCCINATE_SUBSYSTEM_FILTERS = (
    "citrate cycle",
    "tca cycle",
    "tricarboxylic acid",
    "glyoxylate",
    "succinate",
    "transport [c, m]",
    "transport [c, e]",
    "propanoate",
    "pyruvate",
    "cellular response to anaerobic conditions",
)


def collect_succinate_neighbourhood(
    model: Model,
    fluxes: pd.Series | None = None,
    depth: int = 2,
    include_subsystems: Iterable[str] | None = None,
) -> pd.DataFrame:
    visited_reactions = set()
    frontier_metabolites = set(SUCCINATE_METABOLITE_IDS)
    records: List[Dict[str, float]] = []

    if include_subsystems:
        filters = tuple(token.lower() for token in include_subsystems)
    else:
        filters = tuple()

    for _ in range(depth):
        next_frontier = set()
        for met_id in frontier_metabolites:
            metabolite = model.metabolites.get_by_id(met_id)
            for reaction in metabolite.reactions:
                if reaction.id in visited_reactions:
                    continue
                visited_reactions.add(reaction.id)
                subsystem = reaction.subsystem or "Unannotated"
                if filters:
                    subsystem_label = subsystem.lower()
                    if not any(token in subsystem_label for token in filters):
                        continue
                flux_value = (
                    float(fluxes[reaction.id])
                    if fluxes is not None and reaction.id in fluxes.index
                    else float("nan")
                )
                abs_flux = abs(flux_value) if not math.isnan(flux_value) else float("nan")
                stoich = reaction.metabolites.get(metabolite, 0.0)
                records.append(
                    {
                        "reaction_id": reaction.id,
                        "name": reaction.name,
                        "subsystem": subsystem,
                        "succinate_stoichiometry": stoich,
                        "flux": flux_value,
                        "abs_flux": abs_flux,
                    }
                )
                for met in reaction.metabolites:
                    if met.id not in SUCCINATE_METABOLITE_IDS:
                        next_frontier.add(met.id)
        frontier_metabolites = next_frontier
    columns = [
        "reaction_id",
        "name",
        "subsystem",
        "succinate_stoichiometry",
        "flux",
        "abs_flux",
    ]
    table = pd.DataFrame(records, columns=columns)
    if table.empty:
        return table.reset_index(drop=True)
    if fluxes is not None:
        table = table.sort_values("abs_flux", ascending=False)
    else:
        table = table.sort_values("reaction_id")
    return table.reset_index(drop=True)


def summarize_subsystems(table: pd.DataFrame) -> pd.DataFrame:
    if table.empty:
        return pd.DataFrame({"subsystem": [], "reaction_count": []})
    counts = table.groupby("subsystem")["reaction_id"].count().sort_values(ascending=False)
    return counts.rename("reaction_count").reset_index()


def plot_succinate_network(
    model: Model,
    table: pd.DataFrame,
    fluxes: pd.Series | None = None,
    max_reactions: int = 20,
) -> None:
    if table.empty:
        print("Succinate neighbourhood filter returned no reactions. Increase the depth or broaden the subsystem filters.")
        return
    selected = table.head(max_reactions)
    if fluxes is not None and "abs_flux" in table.columns:
        selected = table.nlargest(max_reactions, "abs_flux")
    reaction_ids = selected["reaction_id"].tolist()
    graph = nx.DiGraph()
    for rxn_id in reaction_ids:
        rxn = model.reactions.get_by_id(rxn_id)
        graph.add_node(
            rxn_id,
            kind="reaction",
            label=rxn.id,
            flux=float(fluxes[rxn_id]) if fluxes is not None and rxn_id in fluxes.index else 0.0,
        )
        for met, coeff in rxn.metabolites.items():
            label = f"{met.name} [{met.compartment}]"
            if met.id in SUCCINATE_METABOLITE_IDS:
                label = f"Succinate [{met.compartment}]"
            graph.add_node(met.id, kind="metabolite", label=label)
            if coeff < 0:
                graph.add_edge(met.id, rxn_id, weight=abs(coeff))
            else:
                graph.add_edge(rxn_id, met.id, weight=abs(coeff))
    pos = nx.spring_layout(graph, seed=42)
    reaction_nodes = [n for n, attrs in graph.nodes(data=True) if attrs.get("kind") == "reaction"]
    metabolite_nodes = [n for n, attrs in graph.nodes(data=True) if attrs.get("kind") == "metabolite"]
    flux_values = [abs(graph.nodes[n].get("flux", 0.0)) for n in reaction_nodes]
    cmap = plt.cm.viridis
    if flux_values and max(flux_values) > 0:
        reaction_colors = [cmap(val / max(flux_values)) for val in flux_values]
    else:
        reaction_colors = [cmap(0.0) for _ in reaction_nodes]
    metabolite_colors = ["#f4b942" if n in SUCCINATE_METABOLITE_IDS else "#6baed6" for n in metabolite_nodes]
    fig, ax = plt.subplots(figsize=(12, 8))
    nx.draw_networkx_nodes(
        graph,
        pos,
        nodelist=metabolite_nodes,
        node_color=metabolite_colors,
        node_shape="o",
        node_size=900,
        ax=ax,
    )
    nx.draw_networkx_nodes(
        graph,
        pos,
        nodelist=reaction_nodes,
        node_color=reaction_colors,
        node_shape="s",
        node_size=700,
        ax=ax,
    )
    nx.draw_networkx_edges(graph, pos, arrowstyle="-|>", arrowsize=12, edge_color="#555555", width=1.5, ax=ax)
    labels = {node: attrs["label"] for node, attrs in graph.nodes(data=True)}
    nx.draw_networkx_labels(graph, pos, labels=labels, font_size=8, ax=ax)
    ax.set_title("Succinate-centred reaction network (top flux routes)")
    ax.axis("off")
    if flux_values and max(flux_values) > 0:
        norm = plt.Normalize(vmin=0, vmax=max(flux_values))
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label("Relative pFBA flux (|mmol gDW⁻¹ h⁻¹|)")
    plt.show()


In [ ]:
# Select the best succinate scenario overall for detailed mapping
best_idx = sweep_results[sweep_results['status'] == 'optimal']['succinate_flux'].idxmax()
if pd.notna(best_idx):
    best_row = sweep_results.loc[best_idx]
    print('Best scenario:')
    display(best_row[['feed_label', 'glucose_limit', 'oxygen_limit', 'proton_limit', 'temperature_tag', 'succinate_flux', 'biomass_flux']])

    with analysis_model:
        apply_kennedy_medium(analysis_model, glucose_limit=best_row['glucose_limit'])
        apply_feed_profile(analysis_model, FEED_PROFILES[best_row['feed_label']])
        configure_environment(
            analysis_model,
            oxygen_limit=best_row['oxygen_limit'],
            proton_limit=best_row['proton_limit'],
            ammonium_limit=best_row['ammonium_limit'],
            temperature_tag=best_row['temperature_tag'],
        )
        analysis_model.objective = analysis_model.reactions.get_by_id(SUCCINATE_EXCHANGE_ID)
        best_solution = pfba(analysis_model)
        neighbourhood = collect_succinate_neighbourhood(
            analysis_model,
            fluxes=best_solution.fluxes,
            depth=2,
            include_subsystems=KEY_SUCCINATE_SUBSYSTEM_FILTERS,
        )
        display(neighbourhood[['reaction_id', 'name', 'subsystem', 'flux']].head(20))
        display(summarize_subsystems(neighbourhood))
        plot_succinate_network(analysis_model, neighbourhood, fluxes=best_solution.fluxes, max_reactions=18)
else:
    print('No feasible scenario found for succinate optimisation.')


## 8. Next explorations

* Fix a minimum succinate secretion and re-optimise for biomass to explore trade-offs.
* Vary NGAM values to emulate alternative temperature stresses or maintenance demands.
* Export flux distributions for transcriptomic integration or ^13C MFA comparisons.